# Computer Vision – LBP, HOG & Feature Extractors

Un descriptor de características es un algoritmo y una metodología que nos dice cómo se cuantifica localmente una región de entrada de una imagen. Un descriptor de características acepta una única imagen de entrada y devuelve múltiples vectores de características.

En la práctica, solo queremos describir, es decir, extraer características de las regiones salientes o "interesantes" de una imagen. Es común que apliquemos detectores de keypoint para encontrar regiones "interesantes" de una imagen. Estas regiones se usan luego como entrada para nuestros descriptores de características: dado que N regiones interesantes de una imagen deben describirse, recibiremos N vectores de características del descriptor de características. Ejemplos de descriptores de características incluyen SIFT, SURF, ORB, BRISK, BRIEF y FREAK.

Los descriptores de características tienden a ser mucho más poderosos que nuestros descriptores de imagen básicos, ya que toman en cuenta la localidad de las regiones en una imagen y las describen por separado. Los descriptores de características también tienden a ser mucho más robustos a los cambios en la imagen de entrada, como rotación, traslación, rotación) y cambios en el punto de vista.

## Local Binary Descriptor

In [1]:
from __future__ import print_function
from imutils import paths
import numpy as np
import argparse
import cv2

In [2]:
from skimage import feature
import numpy as np

class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        # Calcular la representación del Local Binary Descriptor de la imagen, y luego
        # usar la representación LBP para construir el histograma de patrones
        lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=range(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))

        # Normalizar el histograma
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        # Devolver el histograma de patrones binarios locales.
        return hist

In [3]:
# Inicializaremos el descriptor de Local Binary Descriptor e inicializaremos el diccionario de índice
# donde el nombre de archivo de la imagen es la clave y las características son el valor
desc = LocalBinaryPatterns(24, 8)
index = {}

# loop sobre las imagenes de ropa
for imagePath in paths.list_images("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/lbp/shirts/"):
    # Cargamos la imagen, la convertimos a escala de grises y la describiremos (Histograms)
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = desc.describe(gray)
    
    # Actualizar el diccionario de índice
    filename = imagePath[imagePath.rfind("/") + 1:]
    index[filename] = hist

# Carga de la imagen de consulta y extraemos Local Binary Descriptor de ella
query = cv2.imread("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/lbp/queries/query_01.jpg")
queryFeatures = desc.describe(cv2.cvtColor(query, cv2.COLOR_BGR2GRAY))

# Mostramos la imagen de consulta e inicializamos el diccionario de resultados.
cv2.imshow("Query", query)
results = {}

# loop sobre el Index
for (k, features) in index.items():
    # Calcular la distancia chi-cuadrado entre las características actuales y la características
    # de la consulta, luego actualizar el diccionario de resultados
    d = 0.5 * np.sum(((features - queryFeatures) ** 2) / (features + queryFeatures + 1e-10))
    results[k] = d

# ordenar los resultados
results = sorted([(v, k) for (k, v) in results.items()])[:3]

# loop sobre los resultados
for (i, (score, filename)) in enumerate(results):
    print("#%d. %s: %.4f" % (i + 1, filename, score))
    image = cv2.imread("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/lbp/shirts" + "/" + filename)
    cv2.imshow("Result #{}".format(i + 1), image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#1. shirt_02.jpg: 0.0073
#2. shirt_03.jpg: 0.0119
#3. shirt_04.jpg: 0.0153


## HOG (Histogram Oriented Gradient)

El histograma de gradientes orientados o HOG, son descriptores utilizados principalmente en la visión por computadora y el aprendizaje automático para la detección de objetos. Sin embargo, también podemos usar descriptores HOG para cuantificar y representar tanto la forma como la textura.

1. Normalizar la imagen antes de la descripción(Histogramas).
2. Calcular gradientes en las direcciones x e y.
3. Obtención de votos ponderados en celdas espaciales y de orientación.
4. Contrastar que normaliza las celdas espaciales superpuestas.
5. Recopilación de todos los histogramas de gradientes orientados para formar el vector de características final.

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from skimage import exposure
from skimage import feature
from imutils import paths
import imutils
import cv2
import os

In [5]:
# Inicializamos la matriz de datos y de labels.
print("[INFO] extracting features...")
data = []
labels = []

# loop sobre los caminos de imagen en el conjunto de entrenamiento
for imagePath in paths.list_images("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/hog/car_logos/"):
    # extraer la marca del coche
    print(imagePath)
    filename_w_ext = os.path.basename(imagePath)
    filename, file_extension = os.path.splitext(filename_w_ext)
    filename = filename.split("_")
    print(filename[0])

    # Cargamos la imagen, y la convertimos a escala de grises y detecte los bordes.
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edged = imutils.auto_canny(gray)

    
    # Tenemos que encontrar contornos en el "edge map", manteniendo solo el más grande, ya que
    # se supone que es el logo de la marca
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[1]
    c = max(cnts, key=cv2.contourArea)

    # extraer el logo del auto y cambiar su tamaño a un ancho canónico y altura
    (x, y, w, h) = cv2.boundingRect(c)
    logo = gray[y:y + h, x:x + w]
    logo = cv2.resize(logo, (200, 100))

    # Extraer el HOG(Histogram Oriented Gradients) del logo.
    H = feature.hog(logo, orientations=9, pixels_per_cell=(10, 10),
        cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1")

    # Actualizar los datos y etiquetas
    data.append(H)
    labels.append(filename[0])

# "Train" el clasificador de KNN
print("[INFO] training classifier...")
model = KNeighborsClassifier(n_neighbors=1)
model.fit(data, labels)
print("[INFO] evaluating...")

# loop sobre el conjunto de datos de prueba
for (i, imagePath) in enumerate(paths.list_images("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/hog/test_images")):
    # Cargar la imagen de prueba, convertirla a escala de grises y cambiar su tamaño a
    # el tamaño canónico
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    logo = cv2.resize(gray, (200, 100))

    # Extraer el HOG(Histogram Oriented Gradients) de la imagen de prueba y
    # predecir la marca del coche
    (H, hogImage) = feature.hog(logo, orientations=9, pixels_per_cell=(10, 10),
        cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1", visualize=True)
    pred = model.predict(H.reshape(1, -1))[0]

    # Visualizar la imgen proveniente del HOG
    hogImage = exposure.rescale_intensity(hogImage, out_range=(0, 255))
    hogImage = hogImage.astype("uint8")
    cv2.imshow("HOG Image #{}".format(i + 1), hogImage)

    cv2.putText(image, pred.title(), (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
        (0, 255, 0), 3)
    cv2.imshow("Test Image #{}".format(i + 1), image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[INFO] extracting features...
C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/hog/car_logos/audi\audi_01.png
audi
C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/hog/car_logos/audi\audi_02.png
audi
C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/hog/car_logos/audi\audi_03.png
audi
C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/hog/car_logos/audi\audi_04.png
audi
C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/hog/car_logos/audi\audi_05.png
audi
C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/hog/car_logos/ford\ford_01.png
ford
C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/hog/car_logos/ford\ford_02.png
ford
C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/hog/car_logos/ford\ford_03.png
ford
C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/image

## Feature extractor and keypoints

In [7]:
import cv2
import imutils
import numpy as np

## DoG

In [8]:
# Cargar la imagen y convertirla a escala de grises
image = cv2.imread("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/supper.jpg")
orig = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

if imutils.is_cv2():
    detector = cv2.FeatureDetector_create("SIFT")
    kps = detector.detect(gray)

else:
    detector = cv2.xfeatures2d.SIFT_create()
    (kps, _) = detector.detectAndCompute(gray, None)

print("# of keypoints: {}".format(len(kps)))

# Loop de los puntos clave y mostrarlos
for kp in kps:
    r = int(0.5 * kp.size)
    (x, y) = np.int0(kp.pt)
    cv2.circle(image, (x, y), r, (0, 255, 255), 2)

cv2.imshow("Images", np.hstack([orig, image]))
cv2.waitKey(0)
cv2.destroyAllWindows()

# of keypoints: 1082


## SIFT

In [9]:
# Cargar la imagen y convertirla a escala de grises
image = cv2.imread("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/supper.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

if imutils.is_cv2():
    # Inicializar el detector de "Keypoints" y el "local invariant descriptor".
    detector = cv2.FeatureDetector_create("SIFT")
    extractor = cv2.DescriptorExtractor_create("SIFT")

    # detectar "Keypoints" y luego extraer los "local invariant descriptor"
    kps = detector.detect(gray)
    (kps, descs) = extractor.compute(gray, kps)

else:
    # Inicializar el detector de "Keypoints"
    detector = cv2.xfeatures2d.SIFT_create()

    # detectar "Keypoints" y luego extraer los "local invariant descriptor"
    (kps, descs) = detector.detectAndCompute(gray, None)

# Mostrar el array de "Keypoints" y de "local invariant descriptor".
print("[INFO] # of keypoints detected: {}".format(len(kps)))
print("[INFO] feature vector shape: {}".format(descs.shape))

[INFO] # of keypoints detected: 1082
[INFO] feature vector shape: (1082, 128)


## Fast-Hessian

In [10]:
# Cargar la imagen y convertirla a escala de grises
image = cv2.imread("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/supper.jpg")
orig = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

if imutils.is_cv2():
    detector = cv2.FeatureDetector_create("SURF")
    kps = detector.detect(gray)

else:
    detector = cv2.xfeatures2d.SURF_create()
    (kps, _) = detector.detectAndCompute(gray, None)

print("# of keypoints: {}".format(len(kps)))

# Loop sobre los "Keypoints" y mostrarlos
for kp in kps:
    r = int(0.5 * kp.size)
    (x, y) = np.int0(kp.pt)
    cv2.circle(image, (x, y), r, (0, 255, 255), 2)

cv2.imshow("Images", np.hstack([orig, image]))
cv2.waitKey(0)
cv2.destroyAllWindows()

# of keypoints: 2115


## SURF

In [11]:
# Cargar la imagen y convertirla a escala de grises
image = cv2.imread("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/supper.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

if imutils.is_cv2():
    # Inicializar el detector de "Keypoints" y el "local invariant descriptor".
    detector = cv2.FeatureDetector_create("SURF")
    extractor = cv2.DescriptorExtractor_create("SURF")

    # detectar "Keypoints" y luego extraer los "local invariant descriptor"
    kps = detector.detect(gray)
    (kps, descs) = extractor.compute(gray, kps)

else:
    # Inicializar el detector de "Keypoints"
    detector = cv2.xfeatures2d.SURF_create()

    # detectar "Keypoints" y luego extraer los "local invariant descriptor"
    (kps, descs) = detector.detectAndCompute(gray, None)

# Mostrar el array de "Keypoints" y de "local invariant descriptor".
print("[INFO] # of keypoints detected: {}".format(len(kps)))
print("[INFO] feature vector shape(rows, columns): {}".format(descs.shape))

[INFO] # of keypoints detected: 2115
[INFO] feature vector shape(rows, columns): (2115, 64)


## Real Featured Matching

In [12]:
from __future__ import print_function
import numpy as np
import argparse
import cv2
from imutils.feature.factories import FeatureDetector_create, DescriptorExtractor_create, DescriptorMatcher_create

In [13]:
detector = "SURF"
extractor = "SIFT"
visualize = "Yes"

# inicializar el detector de características
# como "DOG" o "FASTHESSIAN"
if detector == "DOG":
    detector = FeatureDetector_create("SIFT")
elif detector == "FASTHESSIAN":
    detector = FeatureDetector_create("SURF")
else:
    detector = FeatureDetector_create(detector)

# Inicializar el extractor de características.
extractor = DescriptorExtractor_create(extractor)

# Inicializar el "matcher" de los keypoints
matcher = DescriptorMatcher_create("BruteForce")

# load the two images and convert them to grayscale
imageA = cv2.imread("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/ironman.jpg")
imageB = cv2.imread("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/iron-man.jpg")
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# detectar "keypoints" en las dos imágenes
kpsA = detector.detect(grayA)
kpsB = detector.detect(grayB)

# extraer características de cada una de las regiones de "keypoints" en las imágenes
(kpsA, featuresA) = extractor.compute(grayA, kpsA)
(kpsB, featuresB) = extractor.compute(grayB, kpsB)

# Hacer que coincidan con los puntos clave o "keypoints" utilizando la distancia euclidiana e inicializar
# la lista de coincidencias reales
rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
matches = []

if rawMatches is not None:
    # Loop sobre los matches en bruto
    for m in rawMatches:
        # Asegurar que la distancia pase la prueba de relación de David Lowe.
        if len(m) == 2 and m[0].distance < m[1].distance * 0.8:
            matches.append((m[0].trainIdx, m[0].queryIdx))

    # mostrar información del diagnóstico
    print("# of keypoints from first image: {}".format(len(kpsA)))
    print("# of keypoints from second image: {}".format(len(kpsB)))
    print("# of matched keypoints: {}".format(len(matches)))

    # Inicializar la imagen de visualización de salida.
    (hA, wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB

    # loop sobre los matches
    for (trainIdx, queryIdx) in matches:
        # Generar un color al azar y dibuja el "match".
        color = np.random.randint(0, high=255, size=(3,))
        color = tuple(map(int, color))
        ptA = (int(kpsA[queryIdx].pt[0]), int(kpsA[queryIdx].pt[1]))
        ptB = (int(kpsB[trainIdx].pt[0] + wA), int(kpsB[trainIdx].pt[1]))
        cv2.line(vis, ptA, ptB, color, 2)

        if visualize == "Each":
            cv2.imshow("Matched", vis)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

    if visualize == "Yes":
        cv2.imshow("Matched", vis)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# of keypoints from first image: 3077
# of keypoints from second image: 1480
# of matched keypoints: 17
